# 1. Load Environment & Huggingface, W&B login

In [1]:
import wandb
import os
os.environ["WANDB_PROJECT"]="QLoRA_Instruction_finetune"

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aeolian83. Use `wandb login --relogin` to force relogin


True

In [2]:
from datasets import load_dataset
from huggingface_hub import login
from dotenv import load_dotenv

load_dotenv()


login(token= os.environ["HF_TOKEN"])

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/aeolian83/.cache/huggingface/token
Login successful


# 2. Dataset Load

- 이 시스템에서는 작업을 외장하드에서 하고 있기 때문에 캐쉬폴더를 별도로 지정(In this system, tasks are being performed on an external hard drive, so a separate cache folder is specified.)

In [3]:
ko_instruction_01 = load_dataset("squarelike/OpenOrca-gugugo-ko", cache_dir="/mnt/t7/.cache/huggingface/datasets")

In [4]:
ko_instruction_01 = ko_instruction_01.shuffle(seed=2160)

In [5]:
ko_instruction_01

DatasetDict({
    train: Dataset({
        features: ['question', 'system_prompt', 'response', 'id'],
        num_rows: 2240294
    })
})

In [6]:
ko_instruction_01 = ko_instruction_01['train'].train_test_split(test_size=0.02)

In [7]:
ko_instruction_01

DatasetDict({
    train: Dataset({
        features: ['question', 'system_prompt', 'response', 'id'],
        num_rows: 2195488
    })
    test: Dataset({
        features: ['question', 'system_prompt', 'response', 'id'],
        num_rows: 44806
    })
})

In [8]:
ko_instruction_01['train'][1500]['question']

'질문: "2000년대에 200만 달러 미만의 탐사 비용이 지출되었나요?"  컨텍스트: "미국 달러로 1파운드당 59달러에 이르는 우라늄 매장량의 5.5%는 경제적으로 실행 가능한 매장량으로 추정되며, 3500만 톤은 광물로 분류됩니다(향후 경제적 채굴이 가능한 합리적 전망). 가격은 2003년 5월 10달러/lb에서 2007년 7월 138달러/lb로 상승했습니다. 이로 인해 2005년에는 20억 달러가 지출되었고, 이는 전년 대비 54% 증가한 금액입니다. 이 추세는 2006년까지 이어져 2004년에 비해 250% 이상 증가한 7억 7,400만 달러 이상의 지출이 이루어졌습니다. OECD 원자력 에너지 기구는 2007년 탐사 비용이 2006년과 같을 것으로 예상했습니다."  Answer:\nA:'

In [9]:
ko_instruction_01['train'][1500]['system_prompt']

''

In [10]:
ko_instruction_01['train'][1500]['response']

'2004년에는 200만 달러 미만의 탐사 비용이 지출되었습니다.'

# 3. Model Load

- QLoRA로 finetune을 하기위해 일반 7B 모델을 BitsAndBytes를 통해 양자화(4bit quantization)하여 로드, 훈련속도 향상을 위해 데이터 타입을 fp32가 아니라 fp16으로 로드(차후에는 bf16으로 로드해서 훈련예정)
- To finetune with QLoRA, a standard 7B model is quantized (4-bit quantization) through BitsAndBytes for loading, and to improve training speed, the data type is loaded as fp16 instead of fp32 (with plans to load and train with bf16 in the future).


```python

quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        llm_int8_threshold=6.0,
        llm_int8_has_fp16_weight=False,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
    ),
```

### *Reference

- https://github.com/huggingface/peft/blob/main/examples/fp4_finetuning/finetune_fp4_opt_bnb_peft.py
- https://colab.research.google.com/drive/19AFEOrCI6-bc7h9RTso_NndRwXJRaJ25?usp=sharing

In [11]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, PeftModel, get_peft_model
from trl import SFTTrainer

In [12]:
output_dir = "./results"
model_id = "beomi/llama-2-ko-7b"
device_map = {"": 0}

In [13]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        llm_int8_threshold=6.0,
        llm_int8_has_fp16_weight=False,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
    ),
    device_map=device_map,
    torch_dtype=torch.float16,
    cache_dir="/mnt/t7/.cache/huggingface/models",
)
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir="/mnt/t7/.cache/huggingface/models")

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

## (1) Check Trainable Parameter

- 아래 코드대로 모델을 로드하면 훈련하는 파라미터수를 확인할 수 있다. 

In [14]:
# from peft import prepare_model_for_kbit_training

# model.gradient_checkpointing_enable()
# model = prepare_model_for_kbit_training(model)

In [14]:
def print_trainable_parameters(model):
    """
    Print the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainble%: {100 * trainable_params / all_param}"
    )

### PEFT훈련을 위한 CONFIG설정, 이 부분은 Trainable Parameter를 확인하지 않아도 QLoRA를 위해서는 무소건 해야 한다.("For PEFT training configuration, this step must be done unconditionally for QLoRA, even without checking for Trainable Parameters.")

In [15]:
# peft_config = LoraConfig(
#     r=64,
#     lora_alpha=16,
#     lora_dropout=0.1,
#     inference_mode=False,
#     task_type="CAUSAL_LM",
# )

peft_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "out_proj", "fc1", "fc2"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

In [17]:
# model = get_peft_model(model, peft_config)
# print_trainable_parameters(model)


In [16]:
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1

In [17]:
tokenizer.pad_token = tokenizer.eos_token

# 학습 진행 중 loss가 치솟다가 0.0으로 떨어지는 문제 해결을 위해 사용
tokenizer.padding_side = "right"

In [18]:
def format_instruction(sample):
    system_prompt = f"### System: {sample['system_prompt']}"
    input = f"### Human: {sample['question']}" if len(sample["question"]) > 0 else None
    output = f"### Assistant\n{sample['response']}"
    # join all the parts together
    prompt = "\n\n".join([i for i in [system_prompt, input, output] if i is not None])
    return prompt

# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{format_instruction(sample)}{tokenizer.eos_token}"
    return sample

In [19]:
train_dataset = ko_instruction_01['train'].map(template_dataset, remove_columns=list(ko_instruction_01['train'].features), num_proc=10)

Map (num_proc=10):   0%|          | 0/2195488 [00:00<?, ? examples/s]

In [20]:
test_dataset = ko_instruction_01['test'].map(template_dataset, remove_columns=list(ko_instruction_01['test'].features), num_proc=10)

Map (num_proc=10):   0%|          | 0/44806 [00:00<?, ? examples/s]

In [24]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    save_steps=200,
    save_total_limit=3, # 가장 최근 체크포인트 3개만 저장합니다.
    logging_steps=30,
    report_to="wandb",
    learning_rate=3e-4,
    max_grad_norm=0.3,
    num_train_epochs=1, # epochs 대신 max_steps을 기준으로 할 수 있습니다.
    warmup_ratio=0.03,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit", # paged_adamw_8bit 사용시 메모리를 더 절약할 수 있지만 loss가 0으로 떨어지는 문제가 있습니다.
    group_by_length=True,
    fp16 = False, # 코랩 무료버전에서 실행 시 "True"를 사용하세요
    bf16 = True, # 코랩 무료버전에서 실행 시 "False"를 사용하세요
    lr_scheduler_type="constant",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)

Map:   0%|          | 0/2195488 [00:00<?, ? examples/s]

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [25]:
trainer.train(resume_from_checkpoint=True)


  0%|          | 0/1097744 [00:00<?, ?it/s]

{'loss': 1.2978, 'grad_norm': 0.89453125, 'learning_rate': 0.0003, 'epoch': 0.23}
{'loss': 1.1235, 'grad_norm': 1.703125, 'learning_rate': 0.0003, 'epoch': 0.23}
{'loss': 1.2271, 'grad_norm': 0.71484375, 'learning_rate': 0.0003, 'epoch': 0.23}
{'loss': 1.0699, 'grad_norm': 0.72265625, 'learning_rate': 0.0003, 'epoch': 0.23}
{'loss': 1.3677, 'grad_norm': 0.96484375, 'learning_rate': 0.0003, 'epoch': 0.23}
{'loss': 1.1357, 'grad_norm': 0.83203125, 'learning_rate': 0.0003, 'epoch': 0.23}
{'loss': 1.1649, 'grad_norm': 2.046875, 'learning_rate': 0.0003, 'epoch': 0.23}
{'loss': 1.2373, 'grad_norm': 1.0078125, 'learning_rate': 0.0003, 'epoch': 0.23}
{'loss': 1.2063, 'grad_norm': 2.21875, 'learning_rate': 0.0003, 'epoch': 0.23}
{'loss': 1.2309, 'grad_norm': 1.15625, 'learning_rate': 0.0003, 'epoch': 0.23}
{'loss': 1.0992, 'grad_norm': 0.921875, 'learning_rate': 0.0003, 'epoch': 0.23}
{'loss': 1.1568, 'grad_norm': 1.4140625, 'learning_rate': 0.0003, 'epoch': 0.23}
{'loss': 1.2178, 'grad_norm': 

KeyboardInterrupt: 

In [ ]:
# trainer.model.save_pretrained(output_dir)

### 훈련이 정체 되어 러닝레이트 수정

In [26]:
training_arguments.learning_rate = 2e-6

In [27]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)

trainer.train(resume_from_checkpoint=True)

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


  0%|          | 0/1097744 [00:00<?, ?it/s]

{'loss': 1.4401, 'grad_norm': 0.890625, 'learning_rate': 0.0003, 'epoch': 0.23}
{'loss': 1.1819, 'grad_norm': 1.109375, 'learning_rate': 0.0003, 'epoch': 0.23}
{'loss': 1.2101, 'grad_norm': 0.9453125, 'learning_rate': 0.0003, 'epoch': 0.23}
{'loss': 1.2069, 'grad_norm': 1.796875, 'learning_rate': 0.0003, 'epoch': 0.23}
{'loss': 1.2275, 'grad_norm': 0.81640625, 'learning_rate': 0.0003, 'epoch': 0.23}
{'loss': 1.2486, 'grad_norm': 0.90625, 'learning_rate': 0.0003, 'epoch': 0.23}
{'loss': 1.1266, 'grad_norm': 1.015625, 'learning_rate': 0.0003, 'epoch': 0.23}
{'loss': 1.285, 'grad_norm': 0.80078125, 'learning_rate': 0.0003, 'epoch': 0.23}
{'loss': 1.114, 'grad_norm': 0.95703125, 'learning_rate': 0.0003, 'epoch': 0.23}
{'loss': 1.3299, 'grad_norm': 1.078125, 'learning_rate': 0.0003, 'epoch': 0.23}


KeyboardInterrupt: 

In [ ]:
# trainer.model.save_pretrained(output_dir)

### 허깅페이스에서 훈련 도중 러닝레이트가 수정 안되는 경우가 많다 한다. 
- 스케쥴러 변경을 시도해 보기로 한다. 

In [28]:
training_arguments.lr_scheduler_type="linear"

In [29]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)

trainer.train(resume_from_checkpoint=True)

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


  0%|          | 0/1097744 [00:00<?, ?it/s]

{'loss': 1.4306, 'grad_norm': 0.796875, 'learning_rate': 0.00023748552560031778, 'epoch': 0.23}
{'loss': 1.1129, 'grad_norm': 0.96484375, 'learning_rate': 0.0002374770733961238, 'epoch': 0.23}
{'loss': 1.3266, 'grad_norm': 1.046875, 'learning_rate': 0.00023746862119192983, 'epoch': 0.23}
{'loss': 1.0834, 'grad_norm': 0.71484375, 'learning_rate': 0.00023746016898773585, 'epoch': 0.23}
{'loss': 1.1259, 'grad_norm': 1.25, 'learning_rate': 0.00023745171678354182, 'epoch': 0.23}
{'loss': 1.1209, 'grad_norm': 1.1015625, 'learning_rate': 0.00023744326457934785, 'epoch': 0.23}
{'loss': 1.0383, 'grad_norm': 0.9375, 'learning_rate': 0.00023743481237515387, 'epoch': 0.23}
{'loss': 1.2496, 'grad_norm': 1.015625, 'learning_rate': 0.0002374263601709599, 'epoch': 0.23}
{'loss': 1.2066, 'grad_norm': 0.87890625, 'learning_rate': 0.0002374179079667659, 'epoch': 0.23}
{'loss': 1.1973, 'grad_norm': 0.984375, 'learning_rate': 0.00023740945576257192, 'epoch': 0.23}
{'loss': 1.1831, 'grad_norm': 1.0, 'learni

KeyboardInterrupt: 

In [30]:
training_arguments.learning_rate = 3e-5
training_arguments.lr_scheduler_type="linear"

In [31]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)

trainer.train(resume_from_checkpoint=True)

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


  0%|          | 0/1097744 [00:00<?, ?it/s]

{'loss': 1.1436, 'grad_norm': 1.1796875, 'learning_rate': 0.00023708827200320055, 'epoch': 0.23}
{'loss': 1.1665, 'grad_norm': 1.0234375, 'learning_rate': 0.00023707981979900655, 'epoch': 0.23}
{'loss': 1.1498, 'grad_norm': 1.1953125, 'learning_rate': 0.00023707136759481257, 'epoch': 0.23}
{'loss': 1.2756, 'grad_norm': 1.2109375, 'learning_rate': 0.0002370629153906186, 'epoch': 0.23}
{'loss': 1.1218, 'grad_norm': 0.92578125, 'learning_rate': 0.00023705446318642462, 'epoch': 0.23}
{'loss': 1.3295, 'grad_norm': 1.0546875, 'learning_rate': 0.00023704601098223062, 'epoch': 0.23}
{'loss': 1.1145, 'grad_norm': 0.6171875, 'learning_rate': 0.00023703755877803665, 'epoch': 0.23}
{'loss': 1.1597, 'grad_norm': 1.328125, 'learning_rate': 0.00023702910657384267, 'epoch': 0.23}
{'loss': 1.0881, 'grad_norm': 0.8984375, 'learning_rate': 0.0002370206543696487, 'epoch': 0.23}
{'loss': 1.0414, 'grad_norm': 0.890625, 'learning_rate': 0.0002370122021654547, 'epoch': 0.23}
{'loss': 1.2198, 'grad_norm': 0.78

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/peft/utils/other.py:588: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 5b2d1048-4c45-4224-bd13-1b6fe835b7f3)') - silently ignoring the lookup for the file config.json in beomi/llama-2-ko-7b.
  warnings.warn(
/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in beomi/llama-2-ko-7b - will assume that the vocabulary was not modified.
  warnings.warn(


{'loss': 1.32, 'grad_norm': 0.9296875, 'learning_rate': 0.0002281289355575778, 'epoch': 0.26}
{'loss': 1.0934, 'grad_norm': 0.8046875, 'learning_rate': 0.00022812048335338383, 'epoch': 0.26}
{'loss': 1.1709, 'grad_norm': 0.9765625, 'learning_rate': 0.00022811203114918985, 'epoch': 0.26}
{'loss': 1.1055, 'grad_norm': 1.0234375, 'learning_rate': 0.00022810357894499587, 'epoch': 0.26}
{'loss': 1.0093, 'grad_norm': 0.984375, 'learning_rate': 0.00022809512674080187, 'epoch': 0.26}
{'loss': 1.2739, 'grad_norm': 0.9375, 'learning_rate': 0.0002280866745366079, 'epoch': 0.26}
{'loss': 1.074, 'grad_norm': 0.78515625, 'learning_rate': 0.0002280782223324139, 'epoch': 0.26}
{'loss': 1.15, 'grad_norm': 1.3125, 'learning_rate': 0.00022806977012821992, 'epoch': 0.26}
{'loss': 1.1903, 'grad_norm': 0.90625, 'learning_rate': 0.00022806131792402592, 'epoch': 0.26}
{'loss': 1.1385, 'grad_norm': 1.234375, 'learning_rate': 0.00022805286571983194, 'epoch': 0.26}
{'loss': 1.3337, 'grad_norm': 0.94921875, 'lear

TrainOutput(global_step=1097744, training_loss=0.8568375437861445, metrics={'train_runtime': 702337.3492, 'train_samples_per_second': 3.126, 'train_steps_per_second': 1.563, 'train_loss': 0.8568375437861445, 'epoch': 1.0})

In [32]:
trainer.model.save_pretrained(output_dir)